[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camillepradel/simplon-ner/blob/main/fine_tune_ner.ipynb)

## Setup

In [1]:
!pip install spacy==3.0.6

     |████████████████████████████████| 12.8MB 4.4MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 9.1MB 48.0MB/s 
     |████████████████████████████████| 460kB 24.3MB/s 
     |████████████████████████████████| 122kB 21.2MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=a5f0072872def548d4f4d9f1392c880ab755a87d286696892278ea5a6b13b528
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found e

In [2]:
!python -m spacy download fr_core_news_sm

2021-05-01 22:35:17.072107: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 17.3MB 302kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
!git clone https://github.com/camillepradel/simplon-ner.git
%cd simplon-ner

Cloning into 'simplon-ner'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
/content/simplon-ner


## Training data
File `./train.iob` contains annotated data that we will use to fine tune built-in `fr_core_news_sm` model.

In [4]:
!tail ./train.iob

Le	O
Président	O
de	O
Nantes	B-ORG
Métropole	I-ORG
de	O
Nantes	O
,	O
Jean-Marc	O
AYRAULT	O


The objective of this work is to add more annotated data to this file, so that we get better results after fine tuning in following steps.

To do this, we can either gather unatotated data and manually annotate it, which is cumbersome and time consuming, or we can find smart ways to generate similar annotated data.

One example is to parse web pages in which entities are formally identified. Some websites which can be exploited:
* http://www.trendeo.net/blog/ 
* https://www.digital113.fr/presentation-du-cluster/nos-adherents/
* https://www.occitanie-emploi.fr/categorie-poste/industrie/ 
* https://www.fusacq.com/buzz/fr 
* https://www.boursedirect.fr/fr/actualites/flux/entreprises  


## Preprocessing

In [5]:
# convert IOB train and eval files to spacy binary format
!python -m spacy convert ./train.iob ./ -t spacy -n 1 -c iob
!python -m spacy convert ./eval.iob ./ -t spacy -n 1 -c iob

2021-05-01 22:35:29.033121: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (41 documents): train.spacy
2021-05-01 22:35:33.377215: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (265 documents): eval.spacy


## Fine tuning

In [6]:
!python -m spacy train config.cfg  --paths.train ./train.spacy --paths.dev ./eval.spacy --output ./trained_model

2021-05-01 22:35:38.293236: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Created output directory: trained_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-05-01 22:35:48,575] [INFO] Set up nlp object from config
[2021-05-01 22:35:48,590] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-05-01 22:35:48,590] [INFO] Resuming training for: ['ner', 'tok2vec']
[2021-05-01 22:35:48,604] [INFO] Created vocabulary
[2021-05-01 22:35:48,605] [INFO] Finished initializing nlp object
[2021-05-01 22:35:48,605] [INFO] Initialized pipeline components: []
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00      6.34  

## Evaluation

In [7]:
# evaluate our fine-tuned model
!python -m spacy evaluate ./trained_model/model-best/ ./eval.spacy

2021-05-01 22:40:08.491823: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   64.78
NER R   58.66
NER F   61.57
SPEED   10848


=============================== NER (per type) ===============================

           P       R       F
ORG    78.01   58.66   66.97
PER     0.00    0.00    0.00
LOC     0.00    0.00    0.00
MISC    0.00    0.00    0.00



In [8]:
# compare with built-in fr_core_news_sm model
!python -m spacy evaluate fr_core_news_sm ./eval.spacy

2021-05-01 22:40:20.181739: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using CPU

================================== Results ==================================

TOK      -    
TAG      0.00 
POS      -    
MORPH    -    
LEMMA    -    
UAS      -    
LAS      -    
NER P    32.05
NER R    46.06
NER F    37.80
SENT P   64.90
SENT R   83.02
SENT F   72.85
SPEED    2755 


=============================== NER (per type) ===============================

           P       R       F
MISC    0.00    0.00    0.00
ORG    82.98   46.06   59.24
PER     0.00    0.00    0.00
LOC     0.00    0.00    0.00

